In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

## Check Data 

In [3]:
path = Path(os.getcwd()).parent
rates_ccy = pd.read_csv(path / 'rates_test/data/rates_ccy_data.csv')
spots = pd.read_parquet(path / 'rates_test/data/rates_spot_rate_data.parq')
price = pd.read_parquet(path / 'rates_test/data/rates_price_data.parq')

In [4]:
set(spots['ccy_pair'].unique()) - set(rates_ccy['ccy_pair'].unique())
# Set diff, check if any ccy_pair in the spots is not in the rates_ccy

{'AUDCHF', 'USDCAD', 'USDMXN'}

In [5]:
%timeit
missing_rates = set(spots['ccy_pair'].unique()) - set(rates_ccy['ccy_pair'].unique())
# set assumption that the rates available in spots but not in reference data are not convertible
rates_ccy = pd.concat([rates_ccy, pd.DataFrame(map(lambda x: [x, False, np.nan], missing_rates), columns=rates_ccy.columns)])

merged = spots.merge(rates_ccy, on=['ccy_pair'], how='inner')
merged['timestamp'] = pd.to_datetime(merged['timestamp'])
merged = merged.sort_values('timestamp')

price['timestamp'] = pd.to_datetime(price['timestamp'])
price = price.sort_values('timestamp')

df = pd.merge_asof(price, merged, on=['timestamp'], by=['ccy_pair'], tolerance=pd.Timedelta('6hour'), direction='backward')
df['new_price'] = np.where(df['convert_price'], (df['price'] / df['conversion_factor']) + df['spot_mid_rate'], df['price'])
df[~(df['new_price'].isnull())].to_csv(path / 'full_rates.csv', index=False)
df[(df['new_price'].isnull())].to_csv(path / 'no_nearest_rates.csv', index=False)

In [12]:
display(price)
display(merged)

,timestamp,security_id,price,ccy_pair
index,,,,
5268,2021-12-09 22:05:03.110736,id_961,-1.80,USDDKK
5270,2021-12-09 22:05:03.110736,id_961,-1.30,USDDKK
5272,2021-12-09 22:05:03.117154,id_962,-1.10,USDDKK
5274,2021-12-09 22:05:03.117154,id_962,-0.90,USDDKK
5204,2021-12-09 22:05:03.127579,id_934,-150.00,USDHKD
...,...,...,...,...
2184,2021-12-10 19:59:59.979028,id_419,109.33,USDTRY
3240,2021-12-10 19:59:59.979139,id_624,1.50,USDPLN
3234,2021-12-10 19:59:59.979139,id_624,1.30,USDPLN


,timestamp,ccy_pair,spot_mid_rate,convert_price,conversion_factor
147650,2021-12-08 20:00:01.364272,AUDJPY,81.58000,False,NaN
152933,2021-12-08 20:00:05.491967,GBPNOK,11.75035,True,10.0
85755,2021-12-08 20:00:06.447905,USDSEK,9.03580,True,100.0
147118,2021-12-08 20:00:07.541599,AUDCAD,0.90850,True,100.0
202782,2021-12-08 20:00:07.596916,USDTRY,13.67150,False,NaN
...,...,...,...,...,...
194158,2021-12-10 19:59:58.224322,AUDUSD,0.71700,True,100.0
65260,2021-12-10 19:59:58.225589,EURCZK,25.35840,True,1000.0
16805,2021-12-10 19:59:59.015221,EURRON,4.94930,True,10.0
196705,2021-12-10 19:59:59.325715,USDTRY,13.88840,False,NaN


In [35]:
merged[merged['ccy_pair']=='USDNGN']

,timestamp,ccy_pair,spot_mid_rate,convert_price,conversion_factor
56993,2021-12-09 11:11:31.203574,USDNGN,410.00,True,10.0
50162,2021-12-09 11:20:46.193748,USDNGN,410.00,True,10.0
28671,2021-12-09 12:01:34.065631,USDNGN,410.00,True,10.0
182821,2021-12-09 12:53:46.857275,USDNGN,410.00,True,10.0
190377,2021-12-09 13:50:52.154626,USDNGN,410.00,True,10.0
23510,2021-12-09 13:52:33.558713,USDNGN,410.41,True,10.0
127922,2021-12-09 14:27:01.764404,USDNGN,410.00,True,10.0
63070,2021-12-09 16:59:01.705252,USDNGN,410.00,True,10.0
144322,2021-12-09 18:21:16.390233,USDNGN,410.00,True,10.0
184,2021-12-09 19:23:46.551809,USDNGN,411.00,True,10.0


In [37]:
import duckdb

In [43]:
rates_ccy = duckdb.read_csv(path / 'data/rates_ccy_data.csv')
spots = duckdb.read_parquet((path / 'data/rates_spot_rate_data.parq').as_uri())
price = duckdb.read_parquet((path / 'data/rates_spot_rate_data.parq').as_uri())

In [47]:
spots.join(rates_ccy, condition='ccy_pair', how='left')

┌────────────────────────────┬──────────┬───────────────┬───────────────┬───────────────────┐
│         timestamp          │ ccy_pair │ spot_mid_rate │ convert_price │ conversion_factor │
│          varchar           │ varchar  │    double     │    boolean    │       int64       │
├────────────────────────────┼──────────┼───────────────┼───────────────┼───────────────────┤
│ 2021-12-08 20:05:56.818066 │ EURRSD   │      117.5785 │ false         │              NULL │
│ 2021-12-09 17:20:34.180273 │ USDCNH   │       6.37819 │ true          │               100 │
│ 2021-12-09 19:25:27.575882 │ AUDNZD   │        1.0534 │ true          │              1000 │
│ 2021-12-09 11:23:23.262809 │ EURRON   │        4.9497 │ true          │                10 │
│ 2021-12-10 08:53:21.805817 │ USDCNH   │       6.37381 │ true          │               100 │
│ 2021-12-09 18:59:31.215542 │ EURSEK   │       10.2388 │ true          │               100 │
│ 2021-12-09 13:09:42.170482 │ EURSEK   │      10.25735 │ tr